## ewf-wfp-02-01-04 - Land Surface Temperature Anomalies Time Series

Land Surface Temperature Anomalies Time Series

---

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'Land Surface Temperature Anomalies Time Series'),
                ('abstract', 'Land Surface Temperature Anomalies Time Series'),
                ('id', 'ewf-wfp-02-01-04')])

### <a name="parameter">Parameter Definition 

In [ ]:
N_1 = dict([('id', 'N_1'),
            ('value', 'False'),
            ('title', 'No Aggregation'),
            ('abstract', 'No aggregation')])

In [ ]:
N_3 = dict([('id', 'N_3'),
            ('value', 'True'),
            ('title', '30 Day Aggregation'),
            ('abstract', 'Get a 30 day aggregation')])

In [ ]:
N_6 = dict([('id', 'N_6'),
            ('value', 'False'),
            ('title', '60 Day Aggregation'),
            ('abstract', 'Get a 30 day aggregation')])

In [ ]:
N_9 = dict([('id', 'N_9'),
            ('value', 'False'),
            ('title', '90 Day Aggregation'),
            ('abstract', 'Get a 90 day aggregation')])

In [ ]:
N_12 = dict([('id', 'N_12'),
             ('value', 'False'),
             ('title', '120 Day Aggregation'),
             ('abstract', 'Get a 120 day aggregation')])

In [ ]:
N_15 = dict([('id', 'N_15'),
             ('value', 'False'),
             ('title', '150 Day Aggregation'),
             ('abstract', 'Get a 150 day aggregation')])

In [ ]:
N_18 = dict([('id', 'N_18'),
             ('value', 'False'),
             ('title', '180 Day Aggregation'),
             ('abstract', 'Get a 180 day aggregation')])

In [ ]:
N_27 = dict([('id', 'N_27'),
             ('value', 'False'),
             ('title', '270 Day Aggregation'),
             ('abstract', 'Get a 270 day aggregation')])

In [ ]:
N_36 = dict([('id', 'N_36'),
             ('value', 'False'),
             ('title', '360 Day Aggregation'),
             ('abstract', 'Get a 360 day aggregation')])

In [ ]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON((-179.999 89.999, 179.999 89.999, 179.999 -89.999, -179.999 -89.999, -179.999 89.999))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

In [ ]:
nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value', 'Global'),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

In [ ]:
startdate = dict([('id', 'startdate'),
                  ('value', '2016-01-01T00:00Z'),
                  ('title', 'Start date'),
                  ('abstract', 'Start date')]) 

In [ ]:
enddate = dict([('id', 'enddate'),
                ('value', '2016-01-31T23:59Z'),
                ('title', 'End date'),
                ('abstract', 'End date')])

In [ ]:
indexAgg = dict([('id', 'indexAgg'),
                 ('value', 'better-wfp-02-01-02'),
                 ('title', 'Aggregation user'),
                 ('abstract', 'user to access aggregations catalog'),
                 ('minOccurs', '1')])

In [ ]:
apikeyAgg = dict([('id', 'apikeyAgg'),
                  ('value', ''),
                  ('title', 'Aggregation apikey'),
                  ('abstract', 'apikey to access aggregations catalog'),
                  ('minOccurs', '1')])

In [ ]:
indexLTA = dict([('id', 'indexLTA'),
                 ('value', 'better-wfp-02-01-03'),
                 ('title', 'LTA user'),
                 ('abstract', 'user to access LTAs catalog'),
                 ('minOccurs', '1')])

In [ ]:
apikeyLTA = dict([('id', 'apikeyLTA'),
                  ('value', ''),
                  ('title', 'LTA apikey'),
                  ('abstract', 'apikey to access LTAs catalog'),
                  ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the MDOIS stack of products' identifiers

In [ ]:
input_identifiers = ('dummy')

**Input references**

This is the MODIS stack catalogue references

In [ ]:
input_references = ['dummy']

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = "/workspace/modis"

#### Aux folders

In [ ]:
output_folder = ''

In [ ]:
temp_folder = 'temp'

#### Import Modules

In [ ]:
import os
import shutil

import cioppy

import sys
import string
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.wkt import loads

import pandas as pd
import geopandas as gpd

import datetime

ciop = cioppy.Cioppy()

#### Auxiliary vars

In [ ]:
check_results = False

#### Auxiliary methods

In [ ]:
def rm_cfolder(folder):

    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e) 
    
    
# get metadata from catalog
def get_input_metadata (input_refs, apikeys):
    
    # for each product get metadata
    Result_Prod = []
    
    for index,product_ref in enumerate(input_refs):
        
        for index in apikeys:
            if index in product_ref:
                cat_index = index
                cat_apikey = apikeys[index]
        
        # since the search is by identifier
        Result_Prod.append(ciop.search(end_point = product_ref,params =[],output_fields='self,identifier,startdate,enclosure,startdate,enddate,wkt,title',creds='{}:{}'.format(cat_index,cat_apikey))[0] )
    

    input_metadata = gpd.GeoDataFrame.from_dict(Result_Prod)

    input_metadata['startdate'] = pd.to_datetime(input_metadata['startdate'])
    input_metadata['enddate'] = pd.to_datetime(input_metadata['enddate'])
    
    return input_metadata


   
    
def get_metadata(filepath):

    ds = gdal.Open(filepath)
    projection = ds.GetProjection()
    geotransform = ds.GetGeoTransform()
    no_data_value = ds.GetRasterBand(1).GetNoDataValue()
    data_type = ds.GetRasterBand(1).DataType
    return projection, geotransform, no_data_value, data_type


def get_matrix_list(image_list):
    mat_list = []
    for img in image_list:
        dataset = gdal.Open(img)
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        mat_list.append(product_array)
        dataset = None
        
        print(type(product_array))
    return mat_list


def calc_anomaly(agg_file, LTA_file):
    
    if agg_file and LTA_file:
        
        
        agg_and_LTA = get_matrix_list([agg_file, LTA_file])
        print('Aggregation and LTA converted to matrices')
        
        print(agg_and_LTA[0].dtype)
        print(agg_and_LTA[1].dtype)
        
        anomaly_values = (agg_and_LTA[0] * 1.0) - (agg_and_LTA[1] * 1.0)
        
        print(anomaly_values.dtype)
        
        projection, geotransform, no_data_value, data_type = get_metadata(agg_file)
        
        return anomaly_values, projection, geotransform, no_data_value, data_type
    
    else:
        return None, None, None


def write_output_image(filepath, output_matrix, image_format, data_format, mask=None, output_projection=None, output_geotransform=None, no_data_value=None):
    
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    if mask is not None and mask is not 0:
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
        if no_data_value is not None:
            output_matrix[mask > 0] = no_data_value
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
    
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)

    

def write_anomaly_output(anomaly, output_folder, product_name, first_date, last_date, lta_start_year, lta_end_year, aggregation, mask_no_value, N_value, regionOfInterest, roi_name, projection, geo_transform, no_data_value):
    
    filename = os.path.join(output_folder, product_name + '_Anomaly_' + roi_name + '_N' + str(N_value) + '_' + aggregation + '_' + first_date + '_' + last_date + '_LTA' + str(lta_start_year) + '_' + str(lta_end_year) + '.tif')
    
    write_output_image(filename, anomaly, 'GTiff', gdal.GDT_Float32, mask_no_value, projection, geo_transform, no_data_value)
    
    return filename



def get_formatted_date(date_str):
    date = datetime.datetime.strftime(date_str, '%Y-%m-%dT00:00:00Z')
    return date


def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    first_date = get_formatted_date(first_date)
    last_date = get_formatted_date(last_date)
    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))

#### Auxiliary folders

In [ ]:
if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

#### Workflow

##### Get metadata

In [ ]:
message = 'Getting metadata from catalog' 
ciop.log('INFO', message)

# organize indexes and apikeys in a python dictionary
apikeys = {indexAgg['value']: apikeyAgg['value'], indexLTA['value']: apikeyLTA['value']}

# get input data from catalog
input_metadata = get_input_metadata (input_references, apikeys)

input_metadata_LTA = input_metadata[input_metadata['title'].str.find('LTA') != -1]
input_metadata_Agg = input_metadata[input_metadata['title'].str.find('LTA') == -1]

##### Define pairs

In [ ]:
len(input_metadata_LTA)

In [ ]:
len(input_metadata_Agg)

In [ ]:
agg_lat_idx = {}

# N time steps
nlist = [N_1['value'], N_3['value'], N_6['value'], N_9['value'], N_12['value'], N_15['value'], N_18['value'], N_27['value'], N_36['value']]
nlist = [n == 'True' for n in nlist]
nvalues = [1, 3, 6, 9, 12, 15, 18, 27, 36]

for bl,nv in zip(nlist, nvalues):
    
    # only works for selected N time steps
    if bl:
        
        subN_input_metadata_Agg = input_metadata_Agg[input_metadata_Agg['title'].str.contains('_N(?:{})_'.format(nv))]
        subN_input_metadata_LTA = input_metadata_LTA[input_metadata_LTA['title'].str.contains('_N(?:{})_'.format(nv))]

        for index_agg, row_agg in subN_input_metadata_Agg.iterrows():
            #print(row['c1'], row['c2'])
            agg_lat_idx[index_agg] = []

            sm = pd.to_datetime(row_agg['startdate']).month
            sd = pd.to_datetime(row_agg['startdate']).day
            em = pd.to_datetime(row_agg['enddate']).month
            ed = pd.to_datetime(row_agg['enddate']).day



            for index_lta, row_lta in subN_input_metadata_LTA.iterrows():

                if sm == pd.to_datetime(row_lta['startdate']).month and sd == pd.to_datetime(row_lta['startdate']).day and em == pd.to_datetime(row_lta['enddate']).month and ed == pd.to_datetime(row_lta['enddate']).day:
                    if row_agg['title'].split('_')[3] in row_lta['title']:
                        agg_lat_idx[index_agg].append(index_lta)

In [ ]:
for agg_idx in agg_lat_idx:
    print((input_metadata_Agg.loc[agg_idx]['title']) , (input_metadata_LTA.loc[agg_lat_idx[agg_idx][0]]['title']))

In [ ]:
region_of_interest = regionOfInterest['value']
name_of_region = nameOfRegion['value']

In [ ]:
# TODO: add something to choose between LTAs list
for agg_idx in agg_lat_idx:
    #print((input_metadata_Agg.loc[agg_idx]['title']) , (input_metadata_LTA.loc[agg_lat_idx[agg_idx][0]]['title']))
    
    if len(agg_lat_idx[agg_idx]) < 1:
        continue
    
    sub_input_metadata_Agg = input_metadata_Agg.loc[agg_idx]
    sub_input_metadata_LTA = input_metadata_LTA.loc[agg_lat_idx[agg_idx][0]]
    
    
    # get data paths from catalog metadata
    filepath_agg = os.path.join(data_path, sub_input_metadata_Agg['enclosure'].split('/')[-1])
    filepath_LTA = os.path.join(data_path, sub_input_metadata_LTA['enclosure'].split('/')[-1])
    
    
    print(filepath_agg)
    print(filepath_LTA)
    
    
    # get metadata from catalog metadata (Agg and LTA)

    # Agg

    file_name_elements = os.path.basename(filepath_agg).split('.')[0].split('_')

    agg_type = file_name_elements[-3]
    
    Nn = file_name_elements[-4]
    nv = int(Nn.split('N')[-1])

    first_date = sub_input_metadata_Agg['startdate'].strftime('%Y-%m-%d')
    last_date = sub_input_metadata_Agg['enddate'].strftime('%Y-%m-%d')

    print(nv)
    print(first_date)
    print(last_date)
    
    
    # LTA

    file_name_elements = os.path.basename(filepath_LTA).split('.')[0].split('_')

    agg_type_LTA = file_name_elements[-5]
    Nn_LTA = file_name_elements[-6]


    start_year = str(sub_input_metadata_LTA['startdate'].year)
    end_year = str(sub_input_metadata_LTA['enddate'].year)

    print(start_year)
    print(end_year)
    
      

    message = 'Computing Anomaly'
    ciop.log('INFO', message)

    anomaly_values, projection, geotransform, no_data_value, data_type = calc_anomaly(filepath_agg, filepath_LTA)

    message = 'Writing anomaly image'
    ciop.log('INFO', message)

    filename = write_anomaly_output(anomaly_values, output_folder, 'LST', first_date, last_date, start_year, end_year, agg_type, None, nv, region_of_interest, name_of_region, projection, geotransform, no_data_value)
    write_properties_file(filename, datetime.datetime.strptime(first_date, "%Y-%m-%d").date(), datetime.datetime.strptime(last_date, "%Y-%m-%d").date(), region_of_interest)


In [ ]:
if check_results:

    import matplotlib
    import matplotlib.pyplot as plt

    fig = plt.figure()
    plt.imshow(anomaly_values)
    plt.show()

#### Remove temporay files and folders

In [ ]:
rm_cfolder(temp_folder)

os.rmdir(temp_folder)